In [1]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nico\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
data_path = "../data/txt/"

# Choix d'une décennie et du nombre de clusters

In [4]:
DECADE = '1880'
N_CLUSTERS = 9

# Chargement des fichiers de la décennie

In [5]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [6]:
# Exemple de fichiers
files[:5]

['Bxl_1880_Tome_I1_Part_1.txt',
 'Bxl_1880_Tome_I1_Part_10.txt',
 'Bxl_1880_Tome_I1_Part_2.txt',
 'Bxl_1880_Tome_I1_Part_3.txt',
 'Bxl_1880_Tome_I1_Part_4.txt']

In [8]:
texts = [open(data_path + f, encoding='utf-8').read() for f in files]

In [9]:
# Exemple de textes
texts[0][:400]

'VILLE\n\nD E\n\nBULLETIN\n\nBRUXELLES.\n\nCOMMUNAL\n\nA N N É E\n\n1880.\n\nPREMIÈRE PARTIE.\n\nCOMPTE\n\nRENDU\n\nDES\n\nSÉANCES.\n\nBBUXELLES,\n\nIMPRIMERIE DE V JULIEN BAERTSOEN,\nGrand\'Place, 5.\ne\n\n\x0c]\n\nI\n\n£\n\ni\n\n!\n\n\x0cN " 1.\n\nCOMPTE RENDU DE LA SÉANCE DÛ 1 2 JANVIER 1 8 8 0 .\n\nVILLE\n\nDE\n\nBRUXELLES.\n\nBULLETIN\n\nC O M M U N A L\n\nANNÉE\n\nC O N S E I L\n\n1880.\n\nC O M M U N A L .\n\nSéance du 42 Janvier 1880.\nPrésidence de M . F . V'

# Vectorisation du texte

In [10]:
def process_text(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

In [11]:
vectorizer = TfidfVectorizer(tokenizer=process_text,
                                stop_words=stopwords.words('french'),
                                max_df=0.5,
                                min_df=0.1,
                                lowercase=True)

In [12]:
%time tfidf_vectors = vectorizer.fit_transform(texts)

Wall time: 1min 8s


In [13]:
tfidf_vectors

<270x7997 sparse matrix of type '<class 'numpy.float64'>'
	with 461792 stored elements in Compressed Sparse Row format>

In [14]:
# Exemple de vecteur TFIDF
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names()
    ).sort_values(ascending=False)

delecosse        0.338644
délégation       0.206433
trappeniers      0.199561
veldekens        0.179754
monte-charges    0.172714
                   ...   
lout             0.000000
louée            0.000000
louées           0.000000
loués            0.000000
#                0.000000
Length: 7997, dtype: float64

# Clustering des vecteurs TFIDF

In [15]:
km_model = KMeans(n_clusters=N_CLUSTERS)

In [16]:
km_model.fit(tfidf_vectors)

KMeans(n_clusters=9)

In [17]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(km_model.labels_):
    clustering[label].append(files[idx])

In [18]:
pprint(dict(clustering))

{0: ['Bxl_1885_Tome_I2_Part_10.txt',
     'Bxl_1885_Tome_I2_Part_6.txt',
     'Bxl_1885_Tome_I2_Part_9.txt',
     'Bxl_1886_Tome_I1_Part_1.txt',
     'Bxl_1886_Tome_I1_Part_2.txt',
     'Bxl_1886_Tome_I1_Part_4.txt',
     'Bxl_1886_Tome_I1_Part_5.txt',
     'Bxl_1886_Tome_I1_Part_7.txt',
     'Bxl_1886_Tome_I1_Part_8.txt',
     'Bxl_1886_Tome_I1_Part_9.txt',
     'Bxl_1886_Tome_I2_Part_1.txt',
     'Bxl_1886_Tome_I2_Part_10.txt',
     'Bxl_1886_Tome_I2_Part_12.txt',
     'Bxl_1886_Tome_I2_Part_13.txt',
     'Bxl_1886_Tome_I2_Part_3.txt',
     'Bxl_1886_Tome_I2_Part_5.txt',
     'Bxl_1886_Tome_I2_Part_6.txt',
     'Bxl_1886_Tome_I2_Part_7.txt',
     'Bxl_1886_Tome_I2_Part_8.txt',
     'Bxl_1886_Tome_I2_Part_9.txt',
     'Bxl_1887_Tome_I2_Part_5.txt',
     'Lkn_1883_Tome_RptAn_Part_6.txt',
     'Lkn_1885_Tome_I_Part_8.txt'],
 1: ['Lkn_1880_Tome_I_Part_1.txt',
     'Lkn_1880_Tome_I_Part_2.txt',
     'Lkn_1880_Tome_I_Part_3.txt',
     'Lkn_1880_Tome_I_Part_4.txt',
     'Lkn_1880_Tome_RptAn